In [13]:
import os
import sys
src_dir = os.path.join(os.getcwd(), '..', 'src')
sys.path.append(src_dir)

from utils import GLOBAL, functions

In [14]:
import pandas as pd

In [15]:
proj_df = pd.read_csv(GLOBAL.PROJECTIONS_2021, index_col=0)

In [16]:
#Get dataframes of specific pos
rb_proj_df = proj_df[proj_df['Pos'] == 'RB']
wr_proj_df = proj_df[proj_df['Pos'] == 'WR']
te_proj_df = proj_df[proj_df['Pos'] == 'TE']
qb_proj_df = proj_df[proj_df['Pos'] == 'QB']

In [17]:
#ADP for current year and scoring format
adp_df = pd.read_csv(GLOBAL.ADP_2021_HALF_PPR, index_col=0)

adp_df['ADP RANK'] = adp_df['AVG'].rank()

adp_df_cutoff = adp_df[:100]

replacement_players = {
    'RB': '',
    'WR': '',
    'TE': '',
    'QB': ''
}

In [18]:
for _, row in adp_df_cutoff.iterrows():
    position = row['POS'][:2]
    player = row['Player']

    if position in replacement_players:
        replacement_players[position] = player

In [19]:
vor_df = proj_df[['Player', 'Pos', 'Tm', 'Half PPR']]

In [20]:
replacement_values = {}

for position, player_name in replacement_players.items():
    player = vor_df.loc[vor_df['Player'] == player_name]
    replacement_values[position] = player['Half PPR'].tolist()[0]